## Deployer

In [2]:
%pip install azureml-sdk

  Using cached backports.tempfile-1.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached msal-1.32.0-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.3.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached knack-0.12.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached pkginfo-1.12.1.2-py3-none-any.whl.metadata (13 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached paramiko-3.5.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached azure_mgmt_resource-23.3.0-py3-none-any.whl.metadata (41 kB)
  Using cached azure_mgmt_containerregistry-10.3.0-py3-none-any.whl.metadata (23 kB)
  Using cached azure_mgmt_keyvault-10.3.1-py3-none-any.whl.metadata (15 kB)
  Using cached azure_mgmt_authorization-4.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached azure_mgmt_network-28.1.0-py3-none-any.whl.metadata (90 kB)
  Using cached azure_graphrbac-0.61.2-py2.py3-none-any.whl.metadat

In [6]:
%pip install azureml
%pip install azureml-core

Note: you may need to restart the kernel to use updated packages.
  Using cached azureml_core-1.60.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached backports.tempfile-1.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached msal-1.32.0-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.3.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached knack-0.12.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached azure_core-1.33.0-py3-none-any.whl.metadata (42 kB)
  Using cached pkginfo-1.12.1.2-py3-none-any.whl.metadata (13 kB)
  Using cached argcomplete-3.6.2-py3-none-any.whl.metadata (16 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached paramiko-3.5.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached azure_mgmt_resource-23.3.0-py3-none-any.whl.metadata (41 kB)
  Using cached azure_mgmt_containerregistry-10.3.0-py3-none-any.whl.metadata (23 kB)
  Using cached azure_mgmt_s

Creando el workspace:

In [14]:
import json
from azureml.core.model import Model
from azureml.core import Workspace

# Cargando el archivo con el ID
id = open('id_conf_c4.json', 'r')
mi = json.load(id)
my_id = mi["id"]

# Creando el workspace llamando el ID
ws = Workspace.create(name="workspace_clase4",
                      subscription_id = my_id,
                      resource_group = "class4",
                      location = "centralindia")

# Registrando el modelo
model_name = "model_regression"
registered_model = Model.register(model_path="model_regression.pkl",
                                  model_name=model_name,
                                  workspace=ws)

Deploying KeyVault with name workspackeyvaultfbf7337c.
Deploying AppInsights with name workspacinsights26d1e7af.
Deployed AppInsights with name workspacinsights26d1e7af. Took 10.36 seconds.
Deploying StorageAccount with name workspacstorage9eb8de532.
Deployed KeyVault with name workspackeyvaultfbf7337c. Took 28.92 seconds.
Deployed StorageAccount with name workspacstorage9eb8de532. Took 32.83 seconds.
Deploying Workspace with name workspace_clase4.
Deployed Workspace with name workspace_clase4. Took 41.33 seconds.
Registering model model_regression


In [15]:
################ score.py ###################

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{model_name}')
  model = joblib.load(model_path)

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)
    
    # Incluyendo el preprocesamiento de datos
    data = data.drop([
      "NameStyle",
      "Title",
      "FirstName",
      "MiddleName",
      "LastName",
      "Suffix",
      "CompanyName",
      "SalesPerson",
      "EmailAdress",
      "PasswordHash",
      "rowguid",
      "PasswordSalt"], axis=1)
    
    data['Phone'] = data['Phone'].str.replace('-', '')
    data['Phone'] = data['Phone'].str.replace(' ', '')
    data['Phone'] = data['Phone'].str.replace('(', '')
    data['Phone'] = data['Phone'].str.replace(')', '')
    data['Phone'] = data['Phone'].str.replace('+', '')
    data['Phone'] = data['Phone'].astype(np.float64)
    
    data = data.fillna(0) # Rellenando los valores nulos con 0
     
    # Realizando la predicción
    no_format_result = model.predict(data).tolist() # Valores predichos sin formato
    
    final_result = pd.to_datetime(no_format_result, unit = 's') # Valores predichos con formato

    return json.dumps(final_result) 
    
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [18]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configurando el entorno de virtual
v_env = Environment("regression_env")
v_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn', 'numpy'])

# Configurando el entorno de inferencia
inference_config = InferenceConfig(
                                environment=v_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) 

# Desplegando el servicio
service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

/var/folders/by/zjps0pld1k3grf79nwzwkw440000gn/T/ipykernel_80064/474990807.py:19: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [19]:
# Esperando a que el servicio esté listo
service.wait_for_deployment()

# Guardando la URI del servicio
scoring_uri = service.scoring_uri

# Guardando la URI en un archivo JSON
scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
